<a href="https://colab.research.google.com/github/laysabelici/apache-beam/blob/main/apache_beam_CoGroupByKey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#INSTALADOR PARA MÁQUINA VIRTUAL, EM MÁQUINA LOCAL NÃO COLOQUE [INTERATIVE]
pip install apache_beam[interative]

In [5]:
#IMPORT BIBLIOTECA
import apache_beam as beam

In [55]:
p1 = beam.Pipeline()

Tempo_Atrasos = (
    p1
    |"Importar dados" >> beam.io.ReadFromText('/content/voos_sample.csv',
                                              skip_header_lines = 1)
    |"Separar por vírgula" >> beam.Map(lambda record: record.split(','))
    #Realiza uma separação estilo booleano, também foi indicado a coluna para a ação.
    #Agora vamos pegar todos os itens maior que 0 da coluna 8, referentes a atrasos.
    #temmos que indicar o tipo de dado da coluna, pois ele trata sempre como str.
    |"Pegar voos atrasados" >> beam.Filter(lambda record: int(record[8]) > 0)
    #Criando uma Key e um Value, utilizando a coluna 4 (destino) e o atraso.
    |"Criar conjunto par" >> beam.Map(lambda record: (record[4], int(record[8])))
    # Soma todas as keys que são iguais, para um value unico
    |"Somar por key" >> beam.CombinePerKey(sum)
)

In [58]:
Qtd_Atrasos = (
    p1
    |"Importar dados" >> beam.io.ReadFromText('/content/voos_sample.csv',
                                              skip_header_lines = 1)
    |"Separar por vírgula" >> beam.Map(lambda record: record.split(','))
    #Realiza uma separação estilo booleano, também foi indicado a coluna para a ação.
    #Agora vamos pegar todos os itens maior que 0 da coluna 8, referentes a atrasos.
    #temmos que indicar o tipo de dado da coluna, pois ele trata sempre como str.
    |"Pegar voos atrasados" >> beam.Filter(lambda record: int(record[8]) > 0)
    #Criando uma Key e um Value, utilizando a coluna 4 (destino) e o atraso.
    |"Criar conjunto par" >> beam.Map(lambda record: (record[4], int(record[8])))
    # Soma todas as keys que são iguais, para um value unico
    #Fornece a quantidade de atrasos por aeroporto
    |"Somar por key" >> beam.combiners.Count.PerKey(sum)
)

In [59]:
tabela_atrasos = (
    #Criando tabela sendo dicionário.
    #Essa estrutura pode ser útil em determinados casos,
    #especialmente se você estiver agrupando ou processando dados
    #que têm uma relação específica entre eles.
    {'Qtd_Atrasos':Qtd_Atrasos, 'Tempo_Atrasos':Tempo_Atrasos}
    #É usada para agrupar esses dados por chave.
    #útil quando precisamos realizar operações de agregação ou processamento
    #em grupos específicos de dados.
    |"Group by" >> beam.CoGroupByKey()
    |"Mostrar dados" >> beam.Map(print)
)

p1.run()

('LAX', {'Qtd_Atrasos': [4], 'Tempo_Atrasos': [92]})
('HNL', {'Qtd_Atrasos': [1], 'Tempo_Atrasos': [15]})
('DFW', {'Qtd_Atrasos': [1], 'Tempo_Atrasos': [95]})
('OGG', {'Qtd_Atrasos': [1], 'Tempo_Atrasos': [138]})
('JFK', {'Qtd_Atrasos': [4], 'Tempo_Atrasos': [220]})
